In [ ]:
# !pip install -U "git+https://github.com/ab7289-tandon-nyu/csgy6953_DeepLearning_Midterm.git"

!git clone -b alex "https://github.com/ab7289-tandon-nyu/csgy6953_DeepLearning_Midterm.git"
!cp -r /content/csgy6953_DeepLearning_Midterm/src/ .

Cloning into 'csgy6953_DeepLearning_Midterm'...
remote: Enumerating objects: 124, done.
remote: Counting objects: 100% (124/124), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 124 (delta 69), reused 89 (delta 40), pack-reused 0
Receiving objects: 100% (124/124), 33.28 KiB | 11.09 MiB/s, done.
Resolving deltas: 100% (69/69), done.


In [ ]:
import torch
import torch.nn as nn
import random
import numpy as np

import copy
import time

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from src.data import get_transformed_data, make_data_loaders
from src.transforms import make_transforms

batch_size = 512

valid_ratio = 0.1
train_data, valid_data, test_data = (
    get_transformed_data(
        make_transforms = make_transforms, 
        valid_ratio = valid_ratio
    )
)

train_iter, valid_iter, test_iter = (
    make_data_loaders(
        train_data,
        valid_data,
        test_data,
        batch_size
    )
)

**Define our Model

In [ ]:
from src.model import ResNet, StemConfig
from src.utils import initialize_parameters, epoch_time

model_architecture = (
    (1, 128, 0.5),
    (2, 128, 0.5),
    (2, 128, 0.5),
    (2, 128, 0.5),
    (2, 196, 0.5),
    (2, 196, 0.5),
)

stem_config = StemConfig(num_channels=128, kernel_size=3, stride=1, padding=1)
model = ResNet(model_architecture, stem_config=stem_config, output_size=10)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [ ]:
from pathlib import Path

path = "/content/drive/MyDrive/School/Tandon MSCS/Classes/CS-GY 6953: Deep Learning/midterm/"
file_path = path + "resnet_alex_48m_dropout.pt"

model_file = Path(file_path)

Need to run a dummy set of data to initialize the lazy modules before we can use torchsummary

In [ ]:
if model_file.exists() and model_file.is_file():
  print("loading saved model")
  # load our previously trained model
  model.load_state_dict(torch.load(model_file))
else:
  print("initializing new model")
  # intialize a new model
  inputs = torch.empty((512, 3, 32, 32))
  inputs.normal_()
  model = model.to(device)
  y = model(inputs.to(device))
  print(y.size())

  model.apply(initialize_parameters)

loading saved model


In [ ]:
print(f"num params: {sum([p.numel() for p in model.parameters() if p.requires_grad]):,}")

num params: 4,845,850


In [ ]:
from src.engine import train_one_epoch, evaluate

best_loss = float('inf')

EPOCHS  = 20
learning_rate = 1e-3
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

if model_file.is_file():
  # if we loaded a previously saved iteration, we want to get the current
  # best loss otherwise we could overwrite our save with a worse model
  loss, acc = evaluate(model.to(device), test_iterator, criterion, device)
  best_loss = loss
  print(f"Prevous best loss: {loss:.4f}, acc: {acc * 100:.2f}%")

Prevous best loss: f0.5841, acc: f81.39%


In [ ]:
for epoch in range(1, EPOCHS+1):
    start = time.time()

    print(f"Epoch {epoch}")
    train_loss, train_acc = train_one_epoch(model, train_iterator, criterion, optimizer, device)
    train_mins, train_secs = epoch_time(start, time.time())

    print(f"\tTrain elapsed: {train_mins}:{train_secs}, loss: {train_loss:.4f}, acc: {train_acc * 100:.2f}%")

    start = time.time()
    val_loss, val_acc = evaluate(model, valid_iterator, criterion, device)
    val_mins, val_secs = epoch_time(start, time.time())

    print(f"\tValidation elapsed: {val_mins}:{val_secs}, loss: {val_loss:.4f}, acc: {val_acc * 100:.2f}%")

    if val_loss < best_loss:
        best_loss = val_loss
        # torch.save(model.state_dict(), "/content/drive/MyDrive/Colab Notebooks/midterm/resnet_alex_485.pt")
        torch.save(model.state_dict(), file_path)

Epoch 1
	Train elapsed: 1:4, loss: 0.5319, acc: 81.06%
	Validation elapsed: 0:2, loss: 0.6503, acc: 79.13%
Epoch 2
	Train elapsed: 1:5, loss: 0.4895, acc: 82.87%
	Validation elapsed: 0:2, loss: 0.5683, acc: 81.44%
Epoch 3
	Train elapsed: 1:5, loss: 0.4579, acc: 83.95%
	Validation elapsed: 0:2, loss: 0.5227, acc: 82.86%
Epoch 4
	Train elapsed: 1:5, loss: 0.4417, acc: 84.40%
	Validation elapsed: 0:2, loss: 0.5241, acc: 83.28%
Epoch 5
	Train elapsed: 1:5, loss: 0.4269, acc: 84.94%
	Validation elapsed: 0:2, loss: 0.5182, acc: 83.30%
Epoch 6
	Train elapsed: 1:5, loss: 0.4081, acc: 85.46%
	Validation elapsed: 0:2, loss: 0.5031, acc: 83.67%
Epoch 7
	Train elapsed: 1:5, loss: 0.3884, acc: 86.29%
	Validation elapsed: 0:2, loss: 0.4962, acc: 83.85%
Epoch 8
	Train elapsed: 1:6, loss: 0.3788, acc: 86.70%
	Validation elapsed: 0:2, loss: 0.4906, acc: 84.54%
Epoch 9
	Train elapsed: 1:5, loss: 0.3597, acc: 87.13%
	Validation elapsed: 0:2, loss: 0.4468, acc: 85.81%
Epoch 10
	Train elapsed: 1:5, loss: 0

## Evaluate the Model  

In [ ]:
# model.load_state_dict(torch.load("/content/drive/MyDrive/Colab Notebooks/midterm/resnet_alex_485.pt"))
model.load_state_dict(torch.load(file_path))
test_loss, test_acc = evaluate(model.to(device), test_iterator, criterion, device)
print(f"Test Loss: {test_loss:.4f}\nTest Accuracy: {test_acc * 100:.2f}%")

Test Loss: 0.4431
Test Accuracy: 85.99%
